In [1]:
import os
import os.path as osp
import glob

import pandas as pd

In [9]:
ROOT = './'

csv_data = glob.glob(f"{osp.join(ROOT, 'data', '*.csv')}")

assert len(csv_data) > 0, "No csv files are found"

print(f"Found {len(csv_data)} csv files")

Found 6 csv files


# Join the CSV files together

In [3]:
AGE_CSV = '/Users/kctey/Desktop/NUS Y4S1/BN4101/UK_Biobank/data/age.csv'

df = None
for filepath in sorted(csv_data)[::-1]:
    if df is None:
        df = pd.read_csv(filepath)
    else:
        tmp_df = pd.read_csv(filepath)
        
        # inner join age dataframe
        if filepath == AGE_CSV:
            how = 'inner'
        else:
            how = 'outer'  # outer join all cognitive tests and use imputation to fill missing values

        df = df.merge(tmp_df, on='eid', how=how)

# Handle missing values

In [4]:
print(f"Data extracted has {len(df)} rows")

# Total 11 cognitive tests, (does not count eid and 21022)
# only keep record if user has taken more than 5 tests (7 non-NaN columns)
df = df.dropna(thresh=7)
df

Data extracted has 125360 rows


,eid,20156-0.0,20157-0.0,20247-0.0,20248-0.0,20159-0.0,20195-0.0,mean_20131,mean_20132,mean_20133,20240-0.0,20191-0.0,21022-0.0
0,1000085,43.202,64.591,0.0,0.0,22.0,22.0,4.500000,1.000000,22838.0,9.0,6.0,51.0
1,1000158,48.845,96.175,3.0,5.0,5.0,17.0,4.500000,5.000000,35523.0,7.0,5.0,60.0
2,1000251,23.656,36.673,0.0,0.0,23.0,24.0,4.500000,4.000000,14591.0,8.0,10.0,53.0
3,1000279,25.344,51.384,0.0,1.0,25.0,25.0,5.666667,3.333333,21993.0,8.0,4.0,54.0
4,1000295,33.183,52.936,0.0,0.0,21.0,21.0,4.500000,6.000000,32468.0,7.0,6.0,57.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
121837,6001089,NaN,NaN,NaN,NaN,NaN,NaN,4.500000,2.000000,14432.0,7.0,5.0,55.0
121838,6009918,NaN,NaN,NaN,NaN,NaN,NaN,4.500000,6.000000,36960.0,4.0,2.0,64.0
121839,6011793,NaN,NaN,NaN,NaN,NaN,NaN,4.500000,2.000000,16046.5,7.0,6.0,66.0
121840,6014311,NaN,NaN,NaN,NaN,NaN,NaN,4.500000,6.500000,64201.0,5.0,4.0,55.0


In [5]:
df.isna().sum()

eid               0
20156-0.0     16011
20157-0.0     16011
20247-0.0     16011
20248-0.0     16011
20159-0.0      3088
20195-0.0      3088
mean_20131     4756
mean_20132     4756
mean_20133     4756
20240-0.0     10230
20191-0.0      1511
21022-0.0         0
dtype: int64

# Data Imputation

In [12]:
df.corr()

,eid,20156-0.0,20157-0.0,20247-0.0,20248-0.0,20159-0.0,20195-0.0,mean_20131,mean_20132,mean_20133,20240-0.0,20191-0.0,21022-0.0
eid,1.000000,-0.001635,-0.006370,-0.002487,-0.001405,-0.000447,0.000875,-0.001698,0.004143,0.007073,-0.002316,0.001570,0.000653
20156-0.0,-0.001635,1.000000,0.605740,0.434226,0.195198,-0.461843,-0.470475,-0.077137,0.120394,0.415023,-0.187843,-0.244139,0.267043
20157-0.0,-0.006370,0.605740,1.000000,0.159488,0.486705,-0.538856,-0.551675,-0.123524,0.215575,0.420025,-0.297603,-0.397221,0.338319
20247-0.0,-0.002487,0.434226,0.159488,1.000000,0.247415,-0.101943,-0.077428,-0.015009,0.043550,0.078594,-0.062555,-0.060765,0.044282
20248-0.0,-0.001405,0.195198,0.486705,0.247415,1.000000,-0.175468,-0.150001,-0.034597,0.081809,0.102429,-0.109746,-0.136878,0.076879
20159-0.0,-0.000447,-0.461843,-0.538856,-0.101943,-0.175468,1.000000,0.957085,0.134692,-0.214128,-0.406952,0.222683,0.337963,-0.427771
20195-0.0,0.000875,-0.470475,-0.551675,-0.077428,-0.150001,0.957085,1.000000,0.137771,-0.217600,-0.423677,0.218996,0.341737,-0.450979
mean_20131,-0.001698,-0.077137,-0.123524,-0.015009,-0.034597,0.134692,0.137771,1.000000,0.017595,0.049106,0.063420,0.090312,-0.122673
mean_20132,0.004143,0.120394,0.215575,0.043550,0.081809,-0.214128,-0.217600,0.017595,1.000000,0.576935,-0.108937,-0.157556,0.172485
mean_20133,0.007073,0.415023,0.420025,0.078594,0.102429,-0.406952,-0.423677,0.049106,0.576935,1.000000,-0.157444,-0.233139,0.315275
